In [1]:
#new machine learning project

In [2]:
#data set https://archive.ics.uci.edu/ml/datasets/Air+Quality#

#air quality dataset

In [1]:
from dateutil.parser import parse 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
df = pd.read_csv('C:/Users/swati/data-happy/AirQualityUCI.csv', parse_dates=['Date'])
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.dtypes
dates = pd.to_datetime(df.Date, cache=True)
times = pd.to_timedelta(df.Time)
datetimes = dates + times
df.dtypes
df['Date_time']=datetimes
df['Date_time']=pd.to_datetime(df.Date_time , format = '%d/%m/%Y %H.%M.%S')
df.head()
data=df.drop(['Date','Time','Date_time'],axis=1)
data.index=df.Date_time
data.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,-34.207524,1048.990061,-159.090093,1.865683,894.595276,168.616971,794.990168,58.148873,1391.479641,975.072032,9.778305,39.485380,-6.837604
std,77.657170,329.832710,139.789093,41.380206,342.333252,257.433866,321.993552,126.940455,467.210125,456.938184,43.203623,51.216145,38.976670
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,0.600000,921.000000,-200.000000,4.000000,711.000000,50.000000,637.000000,53.000000,1185.000000,700.000000,10.900000,34.100000,0.692300
50%,1.500000,1053.000000,-200.000000,7.900000,895.000000,141.000000,794.000000,96.000000,1446.000000,942.000000,17.200000,48.600000,0.976800
75%,2.600000,1221.000000,-200.000000,13.600000,1105.000000,284.000000,960.000000,133.000000,1662.000000,1255.000000,24.100000,61.900000,1.296200
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000,44.600000,88.700000,2.231000


In [5]:
#missing values are marked -200
#replace them with previous values
#idea:Check for similar temperature and fill in the data of that day??
cols = data.columns
for j in cols:
    for i in range(0,len(data)):
       if data[j][i] == -200:
           data[j][i] = data[j][i-1]

In [4]:
#checking stationarity
#from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues
#johan_test_temp = data.drop([ 'CO(GT)'], axis=1)
#coint_johansen(johan_test_temp,-1,1).eig



train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

In [17]:
#normalisation

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
data_scaled = scaler.fit_transform(train)

In [18]:
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

prediction = model_fit.forecast(model_fit.y, steps=len(valid))

C:\Users\swati\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\swati\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:222: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  ' forecasting.', ValueWarning)
C:\Users\swati\anaconda3\lib\site-packages\statsmodels\base\wrapper.py:36: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


In [19]:
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,13):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]
import math
from sklearn.metrics import mean_squared_error
#check rmse
for i in range(len(cols)):
    print('rmse value for', cols[i], 'is : ', math.sqrt(mean_squared_error(pred.iloc[i], valid.iloc[i])))

rmse value for CO(GT) is :  65.43044124575029
rmse value for PT08.S1(CO) is :  91.45700624836923
rmse value for NMHC(GT) is :  88.67131766618051
rmse value for C6H6(GT) is :  97.81108031493238
rmse value for PT08.S2(NMHC) is :  87.53827173327807
rmse value for NOx(GT) is :  166.8184861611798
rmse value for PT08.S3(NOx) is :  63.98911769627513
rmse value for NO2(GT) is :  123.1304697982967
rmse value for PT08.S4(NO2) is :  250.50051618717356
rmse value for PT08.S5(O3) is :  167.49669859242525
rmse value for T is :  94.13288821827865
rmse value for RH is :  71.94172184661886
rmse value for AH is :  96.83930248482231
